### Exploratory Data Analisys

In [1]:
cd ../

c:\Users\eduar\Documents\usp\AMST\ml_timeseries_usp


In [2]:
from utils import read_all_csv_files,correcting_dtypes,stocks_removing_negatives
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.plotting import register_matplotlib_converters
import os

In [3]:
pd.set_option('display.max_columns', None)

#### 1. Extract data

In [4]:
df = read_all_csv_files()

#### 2. Basic dataframe infos

In [5]:
df.shape

(979485, 35)

The shape shows basic infomation related to the size of dataframe. Witch is raltive big, but not too much

In [6]:
df.head(5)

,mes,material,sku,produto,cor,tamanho,ordem_tamanho,construcao,padronagem,marca,decote,detalhe,loja,centro_distribuicao,idade_loja,uf_loja,cidade_loja,regiao_loja,bairro_loja,clima_loja,perfil_loja,porte_loja,estoque_unidades_inicio,estoque_unidades_fim,estoque_unidades,estoque_unidades_mediana,prop_dias_com_estoque,venda_unidades,venda_valor_total,preco_medio_unitario,preco_unitario_min,preco_unitario_max,variabilidade_preco_unitario,prop_dias_com_venda,total_observacoes_vendas
0,2023-02-01,045d7d474e1255fba6a784818a34f3fa,13bac94c7ada49b30f8d61924a768de4,x blusa,branco,p,2,circular,liso,9,decote u,sem manga,02568ece-15d4-40d5-b05a-47069ea100e2,CD2,26.0,SP,SAO PAULO,SUDESTE,APARECIDA,2,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0
0,2023-08-01,5cf422a5550730fb73deb1c1b8b07e4f,9882544ee33a5526755c5932737e4df7,x blusa,branco,g,4,malha,liso,9,decote u,sem manga,02568ece-15d4-40d5-b05a-47069ea100e2,CD2,26.0,SP,SAO PAULO,SUDESTE,APARECIDA,2,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0
1,2023-08-01,5cf422a5550730fb73deb1c1b8b07e4f,9882544ee33a5526755c5932737e4df7,x blusa,branco,g,4,malha,liso,9,decote u,sem manga,126bc1f6-f329-48ba-8b94-40aed7f58c45,CD2,6.0,SP,SAO PAULO,SUDESTE,CENTRO,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0
1,2023-02-01,045d7d474e1255fba6a784818a34f3fa,13bac94c7ada49b30f8d61924a768de4,x blusa,branco,p,2,circular,liso,9,decote u,sem manga,5f617d6a-efeb-4a43-89de-4515564aceed,CD2,11.0,SP,SAO PAULO,SUDESTE,VILA REAL,1,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0
2,2023-08-01,5cf422a5550730fb73deb1c1b8b07e4f,9882544ee33a5526755c5932737e4df7,x blusa,branco,g,4,malha,liso,9,decote u,sem manga,1492be88-5ceb-49c6-98ff-ef90dae05875,CD2,14.0,ES,ESPIRITO SANTO,SUDESTE,EURICO SALLES,3,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0


In [7]:
df.tail(5)

,mes,material,sku,produto,cor,tamanho,ordem_tamanho,construcao,padronagem,marca,decote,detalhe,loja,centro_distribuicao,idade_loja,uf_loja,cidade_loja,regiao_loja,bairro_loja,clima_loja,perfil_loja,porte_loja,estoque_unidades_inicio,estoque_unidades_fim,estoque_unidades,estoque_unidades_mediana,prop_dias_com_estoque,venda_unidades,venda_valor_total,preco_medio_unitario,preco_unitario_min,preco_unitario_max,variabilidade_preco_unitario,prop_dias_com_venda,total_observacoes_vendas
152813,2024-11-01,2bd2f8c04c83bfcdeb2b58fac0c33212,0c57d0c28002f1493e44d35c6be0f12a,blusa,branco,p,2,malha,liso,9,reto,sem manga,fdf39bea-2605-4363-848b-eabff9be6133,CD2,14.0,SP,SAO PAULO,SUDESTE,CERAMICA,1,3,3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.000000,0
152814,2024-11-01,2bd2f8c04c83bfcdeb2b58fac0c33212,0c57d0c28002f1493e44d35c6be0f12a,blusa,branco,p,2,circular,liso,9,reto,sem manga,fdf39bea-2605-4363-848b-eabff9be6133,CD2,14.0,SP,SAO PAULO,SUDESTE,CERAMICA,1,3,3,8.0,5.0,7.400000,8.0,1.0,4.0,159.6,39.9,39.9,39.9,0.0,0.133333,5
152815,2024-11-01,2bd2f8c04c83bfcdeb2b58fac0c33212,0c57d0c28002f1493e44d35c6be0f12a,blusa,branco,p,2,malha,liso,9,reto,sem manga,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,CD1,28.0,PB,PARAIBA,NORDESTE,MANAIRA,3,3,4,5.0,3.0,3.933333,4.0,1.0,2.0,79.8,39.9,39.9,39.9,0.0,0.066667,2
152816,2024-11-01,2bd2f8c04c83bfcdeb2b58fac0c33212,0c57d0c28002f1493e44d35c6be0f12a,blusa,branco,p,2,circular,liso,9,reto,sem manga,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,CD1,28.0,PB,PARAIBA,NORDESTE,MANAIRA,3,3,4,5.0,17.0,6.366667,5.0,1.0,3.0,119.7,39.9,39.9,39.9,0.0,0.066667,2
152817,2024-11-01,2bd2f8c04c83bfcdeb2b58fac0c33212,0c57d0c28002f1493e44d35c6be0f12a,blusa,branco,p,2,malha,liso,9,sem decote,sem manga,fe26d5e6-bfb7-4a55-a6a7-f85cc3cd2147,CD1,28.0,PB,PARAIBA,NORDESTE,MANAIRA,3,3,4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.000000,0


#### Correctring datatypes


In [8]:
df = correcting_dtypes(df)

In [9]:
df.dtypes

mes                             datetime64[ns]
material                                object
sku                                     object
produto                                 object
cor                                     object
tamanho                                 object
ordem_tamanho                            int64
construcao                              object
padronagem                              object
marca                                   object
decote                                  object
detalhe                                 object
loja                                    object
centro_distribuicao                     object
idade_loja                               int64
uf_loja                                 object
cidade_loja                             object
regiao_loja                             object
bairro_loja                             object
clima_loja                              object
perfil_loja                             object
porte_loja   

First view shows key information about date, material,stores and sku. There are a lot of columns with information about product, stocks sales. And there are missing values related to price.

#### 4. Basic stats

In [10]:
df.describe()

,mes,ordem_tamanho,idade_loja,estoque_unidades_inicio,estoque_unidades_fim,estoque_unidades,estoque_unidades_mediana,prop_dias_com_estoque,venda_unidades,venda_valor_total,preco_medio_unitario,preco_unitario_min,preco_unitario_max,variabilidade_preco_unitario,prop_dias_com_venda,total_observacoes_vendas
count,979485,979485.000000,979485.000000,979485.000000,979485.000000,979485.000000,979485.000000,979485.000000,979485.000000,979485.000000,259847.000000,259847.000000,259847.000000,165978.000000,979485.000000,979485.000000
mean,2024-04-22 04:11:25.080016640,3.203220,16.572751,0.932905,0.935735,0.927376,0.915708,0.316369,0.882000,42.093815,53.259734,51.509032,54.238507,2.208921,0.025871,0.798609
min,2023-02-01 00:00:00,1.000000,0.000000,-13.000000,-13.000000,-10.354839,-13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2023-09-01 00:00:00,2.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.900000,29.900000,29.990000,0.000000,0.000000,0.000000
50%,2024-05-01 00:00:00,3.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.900000,39.900000,39.900000,0.000000,0.000000,0.000000
75%,2024-12-01 00:00:00,4.000000,19.000000,1.000000,1.000000,1.100000,1.000000,0.925926,1.000000,29.900000,49.900000,49.900000,49.900000,2.954977,0.032258,1.000000
max,2025-07-01 00:00:00,9.000000,54.000000,69.000000,69.000000,59.548387,61.000000,1.000000,66.000000,4493.770000,179.900000,179.900000,179.900000,98.924239,1.000000,27.000000
std,NaN,1.716345,10.440480,2.005210,2.001553,1.869009,1.916312,0.439082,2.250869,111.835862,37.855781,37.840019,38.171466,4.179983,0.060401,1.866721


The cutoff dates goes from "2023-02-01" to "2025-07-01" (2 and half years). The size columns are centrered in 3 and maximum on 9 witch makes sense for most part of clothes has a standard display ("pp","p","m","g","gg"), in terms of size order should be (1,2,3,4,5) centring in number 3 as a medium size, but exists higher and lower displays pushing the max value to 9. 

In [11]:
df['produto'].unique()

array(['x blusa', 'calca', 'camiseta', 'blusa'], dtype=object)

As commented before, blusa and camiseta has the standard display sizes, but calca has higer display sizes.

the age of the stores looks normal, the higher value is less than enterprise foundation and has stores oppened this year witch makes sense be 0. 

However stocks values has negative integers that need to be transformed to 0. One important aspect of stock's data is a half part of observations are 0, and a quarter of it is 1. So the max value of 69 can be a outlier, and the volumety  of data probably is sku in stores per day.

Sales records distribution is very similar and should be analyse in outlier remove approach.

The prices records are the most problem feature, because there are a lot of missing values ~75%. Probably the best way to deal with it is drop and exctract average values from sales.

5. cleanning

In [12]:
df = stocks_removing_negatives(df)

#### TimeSeries Data Analysis